# Learning String Similarity

The paper [@bilenko2002learning] outlines a process by which we can learn the
edit distance between two strings using minimal training over pairs of
coreferent strings.
Coreferent strings are strings that belong to entity references that refer to
the same real world object.
The idea is that if we fine tune the algorithm that computes the string edit
distance to the business domain where we perform entity resolution, we will be
able to identify coreferent entity references more accurately than if we use
a generic edit distance.
The aforementioned paper references an algorithm [@ristad1998learning] which
learns to compute the Levenshtein distance between two strings from training
data.

This notebook showcases Ristad and Yianilos' algorithm using Milenko's approach
to constructing the training corpus. 

In [1]:
import os
import random

import numpy as np
import polars as pl
from matchescu.matching.similarity import LevenshteinLearner


DATADIR = os.path.abspath("../../data")
CSV_PATH = os.path.join(DATADIR, "cora", "cora.csv")

In [2]:
df = pl.read_csv(CSV_PATH, has_header=False, ignore_errors=True).rename({
    "column_1": "id",
    "column_3": "class",
    "column_4": "author",
    "column_5": "volume",
    "column_6": "title",
    "column_7": "institution",
    "column_8": "venue",
    "column_11": "year"
}).select(pl.col("id", "class", "author", "title", "venue", "year"))
display(df)

id,class,author,title,venue,year
i64,str,str,str,str,str
1,"""blum1993""","""avrim blum, merrick furst, mic…","""cryptographic primitives based…","""in pre-proceedings of crypto '…","""1993"""
2,"""blum1993""","""avrim blum, merrick furst, mic…","""cryptographic primitives based…","""proc. crypto 93,""","""1994"""
3,"""blum1993""","""a. blum, m. furst, m. kearns, …","""cryptographic primitives based…","""crypto,""","""1993"""
4,"""blum1994""","""blum, a., furst, m., jackson, …","""weakly learning dnf and charac…","""proceedings of the 26th annual…","""(1994)."""
5,"""blum1994""","""blum, a., furst, m., jackson, …","""weakly learning dnf and charac…","""in proceedings of the twenty-s…","""(1994)."""
…,…,…,…,…,…
1289,"""schapire1998""","""robert e. schapire and yoram s…","""improved boosting algorithms u…","""in proceedings of the eleventh…","""1998"""
1290,"""schapire""","""schapire, r. e., freund, y., b…","""boosting the margin: a new exp…",null,"""(1998)."""
1291,"""schapire1998mm""","""robert e. schapire and yoram s…","""a system for multiclass multi-…","""unpublished manuscript,""","""1998"""


In [3]:
records = list(df.iter_rows(named=True))
dedupe_data = []
y = []
for i, left_record in enumerate(records):
    for j, right_record in enumerate(records, i+1):
        lclass, rclass = left_record["class"], right_record["class"]
        row = {f"{k}_left": v for k,v in left_record.items()}
        row.update(
            {f"{k}_right": v for k,v in right_record.items()}
        )
        dedupe_data.append(row)
        y.append(int(lclass == rclass))
X = pl.DataFrame(dedupe_data).to_numpy()
y = np.array(y)
display(X, y)

array([[1, 'blum1993',
        'avrim blum, merrick furst, michael kearns, and richard j. lipton.',
        ..., 'cryptographic primitives based on hard learning problems.',
        "in pre-proceedings of crypto '93,", '1993'],
       [1, 'blum1993',
        'avrim blum, merrick furst, michael kearns, and richard j. lipton.',
        ..., 'cryptographic primitives based on hard learning problems.',
        'proc. crypto 93,', '1994'],
       [1, 'blum1993',
        'avrim blum, merrick furst, michael kearns, and richard j. lipton.',
        ..., 'cryptographic primitives based on hard learning problems.',
        'crypto,', '1993'],
       ...,
       [1293, 'singer', 'robert e. schapire yoram singer.', ...,
        'a system for multiclass multi-label text categorization.',
        'unpublished manuscript,', '1998'],
       [1293, 'singer', 'robert e. schapire yoram singer.', ...,
        'improved boosting algorithms using confidence-rated predictions.',
        None, None],
       [

array([1, 1, 1, ..., 0, 1, 1])

In [9]:
from sklearn.model_selection import StratifiedKFold

fold = StratifiedKFold(n_splits=10, shuffle=True)
splits = fold.split(X, y)
datasets = []
for train, test in splits:
    datasets.append((X[train], y[train], X[test], y[test]))

In [21]:
import random
from sklearn.svm import LinearSVC


def train_distance_model(features, target, n_samples=100, epochs=10):
    idx_of_matches, = np.where(target == 1)
    matching_recods = features[idx_of_matches]
    coreferent_values = [(val[3], val[9]) for val in matching_recods]
    corpus = list(random.choices(coreferent_values, k=n_samples))
    return LevenshteinLearner().fit(corpus, epochs)


def train_svm(features, target):
    model = LinearSVC()
    model.fit(features, target)
    return model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from jellyfish import levenshtein_distance


distance_model = train_distance_model(X, y)


def _title_levenshtein(row):
    return levenshtein_distance(row[3], row[9]),


def _title_learned_levenshtein(row):
    return distance_model.compute_distance(row[3], row[9])


stats = []
for X_train, y_train, X_test, y_test in datasets:
    title_train = list(map(_title_levenshtein, X_train))
    model = train_svm(title_train, y_train)
    title_test = list(map(_title_levenshtein, X_test))
    prediction = model.predict(title_test)
    
    learned_title_train = list(map(_title_learned_levenshtein, X_train))
    learned_model = train_svm(learned_title_train, y_train)
    learned_title_test = list(map(_title_learned_levenshtein, X_test))
    learned_prediction = model.predict(title_test)
    
    stats.append({
        "levenshtein precision": precision_score(y_test, prediction),
        "levenshtein recall": recall_score(y_test, prediction),
        "levenshtein f1": f1_score(y_test, prediction),
        "learned levenshtein precision": precision_score(y_test, learned_prediction),
        "learned levenshtein recall": recall_score(y_test, learned_prediction),
        "learned levenshtein f1": f1_score(y_test, learned_prediction),
    })

stats = pl.DataFrame(stats)
display(stats)
display(stats.mean())